# Day 17 扩展：自适应思考级别 - 智能客服案例

## 场景

电商客服系统，根据问题复杂度自动选择思考级别：

| 问题类型 | 思考级别 | 示例 |
|---------|---------|------|
| 简单查询 | MINIMAL | "运费多少？" |
| 常规咨询 | LOW | "如何退货？" |
| 需要分析 | MEDIUM | "对比这两款产品" |
| 复杂投诉 | HIGH | "订单出问题，需要赔偿方案" |

## 1. 环境准备

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("GOOGLE_API_KEY"), "请设置 GOOGLE_API_KEY"

In [2]:
from google.adk.agents import Agent
from google.adk.planners import BuiltInPlanner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import Content, Part

## 2. 模拟电商数据库

In [3]:
# 模拟产品数据
PRODUCTS = {
    "iPhone 15": {"price": 5999, "stock": 50, "rating": 4.8, "category": "手机"},
    "iPhone 15 Pro": {"price": 7999, "stock": 30, "rating": 4.9, "category": "手机"},
    "AirPods Pro": {"price": 1899, "stock": 100, "rating": 4.7, "category": "耳机"},
    "MacBook Air": {"price": 8999, "stock": 20, "rating": 4.9, "category": "电脑"},
}

# 模拟订单数据
ORDERS = {
    "ORD001": {"product": "iPhone 15", "status": "已发货", "shipping": "顺丰", "eta": "明天到达"},
    "ORD002": {"product": "AirPods Pro", "status": "配送中", "shipping": "京东", "eta": "今天18:00"},
    "ORD003": {"product": "MacBook Air", "status": "已签收", "shipping": "顺丰", "eta": "已完成"},
}

# 店铺政策
POLICIES = {
    "shipping_fee": "满99包邮，未满99收取10元运费",
    "return_policy": "7天无理由退货，15天质量问题换货",
    "warranty": "电子产品1年保修，耳机6个月保修",
}

print("✅ 模拟数据已加载")

✅ 模拟数据已加载


## 3. 定义客服工具

In [4]:
def query_product(product_name: str) -> dict:
    """查询产品信息"""
    if product_name in PRODUCTS:
        return {"found": True, "product": product_name, **PRODUCTS[product_name]}
    return {"found": False, "message": f"未找到产品: {product_name}"}


def query_order(order_id: str) -> dict:
    """查询订单状态"""
    if order_id in ORDERS:
        return {"found": True, "order_id": order_id, **ORDERS[order_id]}
    return {"found": False, "message": f"未找到订单: {order_id}"}


def get_policy(policy_type: str) -> dict:
    """获取店铺政策（shipping_fee/return_policy/warranty）"""
    if policy_type in POLICIES:
        return {"policy_type": policy_type, "content": POLICIES[policy_type]}
    return {"available_policies": list(POLICIES.keys())}


def compare_products(product1: str, product2: str) -> dict:
    """对比两个产品"""
    p1 = PRODUCTS.get(product1)
    p2 = PRODUCTS.get(product2)
    if p1 and p2:
        return {
            "comparison": {
                product1: p1,
                product2: p2,
                "price_diff": p2["price"] - p1["price"],
            }
        }
    return {"error": "产品不存在"}


def calculate_refund(order_id: str, reason: str) -> dict:
    """计算退款方案"""
    order = ORDERS.get(order_id)
    if not order:
        return {"error": "订单不存在"}
    
    product = PRODUCTS.get(order["product"], {})
    price = product.get("price", 0)
    
    # 根据原因计算退款
    if "质量" in reason or "损坏" in reason:
        refund = price  # 全额退款
        compensation = 50  # 补偿
    elif "不想要" in reason:
        refund = price * 0.95  # 扣5%手续费
        compensation = 0
    else:
        refund = price
        compensation = 0
    
    return {
        "order_id": order_id,
        "original_price": price,
        "refund_amount": refund,
        "compensation": compensation,
        "total": refund + compensation,
        "reason": reason
    }


TOOLS = [query_product, query_order, get_policy, compare_products, calculate_refund]
print("✅ 工具已定义:", [t.__name__ for t in TOOLS])

✅ 工具已定义: ['query_product', 'query_order', 'get_policy', 'compare_products', 'calculate_refund']


## 4. 问题复杂度分类器

In [5]:
def classify_question(question: str) -> str:
    """
    根据问题内容判断复杂度，返回对应的思考级别
    
    实际生产中可以用 ML 模型，这里用简单规则演示
    """
    question = question.lower()
    
    # HIGH: 复杂投诉、需要计算赔偿
    if any(kw in question for kw in ["投诉", "赔偿", "损坏", "假货", "欺骗", "退款方案"]):
        return "HIGH"
    
    # MEDIUM: 需要对比分析
    if any(kw in question for kw in ["对比", "区别", "哪个好", "推荐", "选择"]):
        return "MEDIUM"
    
    # LOW: 常规咨询
    if any(kw in question for kw in ["怎么", "如何", "退货", "换货", "保修"]):
        return "LOW"
    
    # MINIMAL: 简单查询
    return "MINIMAL"


# 测试分类器
test_questions = [
    "运费多少钱？",
    "怎么退货？",
    "iPhone 15 和 Pro 哪个好？",
    "我的订单收到是坏的，要求赔偿！"
]

print("问题分类测试：")
for q in test_questions:
    level = classify_question(q)
    print(f"  [{level:7}] {q}")

问题分类测试：
  [MINIMAL] 运费多少钱？
  [LOW    ] 怎么退货？
  [MEDIUM ] iPhone 15 和 Pro 哪个好？
  [HIGH   ] 我的订单收到是坏的，要求赔偿！


## 5. 自适应客服 Agent

In [6]:
# Agent 缓存，避免重复创建
AGENT_CACHE = {}

def get_agent(thinking_level: str) -> Agent:
    """获取或创建指定级别的 Agent"""
    if thinking_level not in AGENT_CACHE:
        AGENT_CACHE[thinking_level] = Agent(
            model='gemini-3-flash-preview',
            name=f'support_agent_{thinking_level.lower()}',
            instruction="""你是一个专业的电商客服。

职责：
- 回答产品咨询
- 查询订单状态
- 解答政策问题
- 处理退款投诉

要求：
- 用中文回答
- 态度友好专业
- 回答简洁有用
- 涉及金额要准确""",
            tools=TOOLS,
            planner=BuiltInPlanner(
                thinking_config=types.ThinkingConfig(
                    thinking_level=thinking_level
                )
            ),
        )
        print(f"  📦 创建 [{thinking_level}] Agent")
    return AGENT_CACHE[thinking_level]

print("✅ 自适应 Agent 工厂已就绪")

✅ 自适应 Agent 工厂已就绪


## 6. 智能客服入口

In [7]:
session_service = InMemorySessionService()

async def smart_support(question: str) -> dict:
    """
    智能客服：自动分析问题复杂度，选择合适的思考级别
    
    返回：回答内容 + 使用的思考级别
    """
    # 1. 分类问题
    thinking_level = classify_question(question)
    print(f"🔍 问题分类: [{thinking_level}]")
    
    # 2. 获取对应 Agent
    agent = get_agent(thinking_level)
    
    # 3. 创建会话
    session = await session_service.create_session(
        app_name="smart_support",
        user_id="customer"
    )
    
    runner = Runner(
        agent=agent,
        app_name="smart_support",
        session_service=session_service
    )
    
    # 4. 获取回答
    user_content = Content(role="user", parts=[Part(text=question)])
    
    response = ""
    async for event in runner.run_async(
        user_id="customer",
        session_id=session.id,
        new_message=user_content
    ):
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts'):
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text:
                        response += part.text
    
    return {
        "question": question,
        "thinking_level": thinking_level,
        "response": response
    }

print("✅ 智能客服已就绪")

✅ 智能客服已就绪


## 7. 测试场景

### 场景 1: 简单查询 → MINIMAL

In [8]:
result = await smart_support("运费怎么算？")
print(f"\n💬 回答:\n{result['response']}")

🔍 问题分类: [LOW]
  📦 创建 [LOW] Agent



💬 回答:
关于运费的计算规则如下：

*   订单金额**满 99 元**：免运费。
*   订单金额**未满 99 元**：统一收取 **10 元**运费。

如果您有特定的订单需要查询，欢迎随时提供详情。


### 场景 2: 常规咨询 → LOW

In [9]:
result = await smart_support("我想退货，怎么操作？")
print(f"\n💬 回答:\n{result['response']}")

🔍 问题分类: [LOW]

💬 回答:
您好！为您办理退货，我需要先了解一下相关信息。

通常的退货流程如下：
1. 请提供您的**订单号**。
2. 说明您需要**退货的原因**。

在为您正式提交申请前，您可以先查看我们的**退货政策**。如果您能告知订单号，我可以为您查询该订单是否符合退货条件，并为您计算预计的退款金额。

请问您的订单号是多少？


### 场景 3: 对比分析 → MEDIUM

In [10]:
result = await smart_support("iPhone 15 和 iPhone 15 Pro 有什么区别？哪个更值得买？")
print(f"\n💬 回答:\n{result['response']}")

🔍 问题分类: [MEDIUM]
  📦 创建 [MEDIUM] Agent


/Users/robert/project/jupyter-project/dev/25-Day-Agents-Course-by-Google/.venv/lib/python3.12/site-packages/google/genai/_common.py:651: UserWarning: MEDIUM is not a valid ThinkingLevel
  warnings.warn(f'{value} is not a valid {cls.__name__}')



💬 回答:
iPhone 15 和 iPhone 15 Pro 都是非常优秀的手机，但在性能和配置上存在一些核心差异。以下是为您整理的对比分析：

### 1. 价格与基本信息
*   **iPhone 15**：售价 **5,999元**，好评度 4.8。
*   **iPhone 15 Pro**：售价 **7,999元**，好评度 4.9。
*   **差价**：2,000元。

### 2. 主要区别（核心升级点）
*   **屏幕显示**：Pro 版本支持 **120Hz ProMotion 自适应刷新率**（丝滑流畅）以及常亮显示；标准版为 60Hz。
*   **性能芯片**：Pro 版本搭载 **A17 Pro 芯片**（支持硬件级光线追踪，适合重度游戏）；标准版搭载 A16 仿生芯片。
*   **影像系统**：Pro 版本多出一颗 **3倍光学变焦长焦镜头**，并支持 ProRAW 和 ProRes 视频拍摄；标准版为后置双摄系统。
*   **材质与外观**：Pro 版本采用全新的 **钛金属边框**，机身更轻且坚固，并配有可自定义的“操作按钮”；标准版为铝金属边框。
*   **传输速度**：Pro 版本支持 **USB 3 (最高 10Gb/s)**，标准版为 USB 2。

### 哪个更值得买？
这取决于您的预算和使用需求：

*   **如果您追求极致体验：** 经常玩大型游戏、对手机摄影（尤其是长焦）有较高要求，或者希望屏幕看起来更流畅，**iPhone 15 Pro** 是更好的选择，多出的 2,000 元能带来质的飞跃。
*   **如果您追求性价比：** 日常主要使用社交软件、看视频，对专业摄影没有强烈需求，**iPhone 15** 已绰绰有余。它同样拥有灵动岛设计和出色的 4800 万像素主摄，价格也更亲民。

目前两款机型均有现货，您可以根据自己的偏好进行下单。如有其他疑问，欢迎随时咨询！


### 场景 4: 复杂投诉 → HIGH

In [11]:
result = await smart_support("我的订单 ORD001 收到的手机屏幕有划痕，质量有问题！我要投诉并要求赔偿！")
print(f"\n💬 回答:\n{result['response']}")

🔍 问题分类: [HIGH]
  📦 创建 [HIGH] Agent

💬 回答:
非常抱歉听到您的 iPhone 15（订单号：ORD001）屏幕存在划痕，这确实不符合我们的质量标准，给您带来了不便，我们深表歉意。

针对您反馈的质量问题，我们为您提供以下处理方案：

1. **全额退款 + 补偿**：您可以申请全额退货退款，金额为 **5999元**。同时，为了补偿您的损失，我们将额外提供 **50元** 的现金补偿，总计退回 **6049元**。
2. **质量换货**：根据店铺政策，产品在15天内出现质量问题支持免费换货。如果您依然喜欢这款手机，我们可以为您重新寄送一台全新的 iPhone 15。

请问您更倾向于哪种处理方式？如果您决定退换货，请保持包装完整，我会立即为您发起售后流程。


## 8. 批量测试对比

In [12]:
test_cases = [
    "AirPods Pro 多少钱？",
    "订单 ORD002 到哪了？",
    "MacBook Air 和 iPhone 15 Pro 选哪个？",
    "ORD003 的电脑有质量问题，要求全额退款并赔偿损失！",
]

print("=" * 60)
print("批量测试 - 自适应思考级别")
print("=" * 60)

for q in test_cases:
    print(f"\n📝 问题: {q}")
    result = await smart_support(q)
    print(f"💬 回答: {result['response'][:200]}..." if len(result['response']) > 200 else f"💬 回答: {result['response']}")
    print("-" * 40)

批量测试 - 自适应思考级别

📝 问题: AirPods Pro 多少钱？
🔍 问题分类: [MINIMAL]
  📦 创建 [MINIMAL] Agent


/Users/robert/project/jupyter-project/dev/25-Day-Agents-Course-by-Google/.venv/lib/python3.12/site-packages/google/genai/_common.py:651: UserWarning: MINIMAL is not a valid ThinkingLevel
  warnings.warn(f'{value} is not a valid {cls.__name__}')


💬 回答: AirPods Pro 的售价为 1899 元。目前库存充足，如果您需要下单或了解更多参数，请随时告诉我。
----------------------------------------

📝 问题: 订单 ORD002 到哪了？
🔍 问题分类: [MINIMAL]
💬 回答: 您好！您的订单 ORD002（AirPods Pro）目前正由京东快递配送中。

目前的物流状态为：**配送中**。
预计送达时间：**今天 18:00 前**。

请保持电话畅通，以便快递员联系您。如有其他问题，欢迎随时咨询！
----------------------------------------

📝 问题: MacBook Air 和 iPhone 15 Pro 选哪个？
🔍 问题分类: [MINIMAL]
💬 回答: 这两款产品属于完全不同的类别，选择哪一款主要取决于您的具体需求：

1. **MacBook Air (售价 ¥8,999)**：
   - **定位**：轻薄笔记本电脑。
   - **适用场景**：适合办公、学习、文字处理、轻度视频剪辑及日常娱乐。如果您需要一个高效的生产力工具，它是更好的选择。

2. **iPhone 15 Pro (售价 ¥7,999)**：
   - **定位**：旗舰...
----------------------------------------

📝 问题: ORD003 的电脑有质量问题，要求全额退款并赔偿损失！
🔍 问题分类: [HIGH]
💬 回答: 非常抱歉听到您购买的 MacBook Air (订单号：ORD003) 出现了质量问题，给您带来了不便。

根据我们的售后政策和系统核实，您的全额退款及赔偿申请已处理。具体方案如下：

*   **商品全额退款：** 8999元
*   **补偿金额：** 50元
*   **退款总计：** 9049元

退款将按原支付途径退回。请您保持电话畅通，我们会尽快安排顺丰上门取件。如果您还有其他问题，请...
----------------------------------------


## 9. 总结

### 自适应思考的优势

| 优势 | 说明 |
|------|------|
| 成本优化 | 简单问题用低级别，省 token |
| 响应更快 | 不需要每次都深度思考 |
| 质量保证 | 复杂问题自动升级处理 |

### 实际应用建议

1. **分类器** - 可以用 ML 模型替代规则
2. **监控** - 记录各级别使用比例和效果
3. **反馈** - 根据用户满意度调整分类阈值

### 代码复用模式

```python
# 核心模式：分类 → 选择 Agent → 执行
level = classify_question(question)
agent = get_agent(level)
response = await run(agent, question)
```